## Imports and Modules and Libraries Oh MY

In [ ]:
# 1. Basic Python Imports
import pandas as pd
import numpy as np
import pickle as pi
import os
from pathlib import Path

# 2. API Keys
from dotenv import load_dotenv

# 3. API PULL
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# 4. Panadas_Datareader
import pandas_datareader.data as web
from pandas_datareader import data, wb
import pandas_datareader as pdr

# 5. Date Time
import datetime as dt
from datetime import datetime

# 6. Plotting
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import hvplot.pandas 
import hvplot.dask  

### FUNKY FUNCTIONS

In [ ]:
def cn_close_price_dfer(digital_asset=list):
    
    # Date information 
    startdate = datetime(2018,1,1)
    enddate = datetime.today() 
    
    # API PULL to get Data for coins
    adj_close = web.DataReader(digital_asset, 'yahoo',startdate, enddate)
    adj_close_df = adj_close.filter(like="Adj Close")
    adj_close_df.columns = adj_close_df.columns.droplevel(-2)
    
    return(adj_close_df)
    

In [ ]:
def indx_close_price_xls(xlsfile_path=str):
    
    startdate = datetime(2018,1,1)
    
    
    dfs = pd.DataFrame()
    dfs= pd.read_excel(xlsfile_path, skiprows=6).dropna()
    dfs= dfs.rename({'Effective date ':'Date', 'S&P Cryptocurrency Broad Digital Market Index (USD)':'SPCBDM-USD'}, axis='columns')
    dfs= dfs.set_index('Date')
    dfs=dfs.dropna()
  
    return(dfs)


In [ ]:
def mrkt_cap_api_df():
    
    """
    This function get the market cap data that will be important when it comes
    down to doing calculations for weight allocations
    """
    
    # Bring In Dataframe with marketcap data for our coins
    # Using the Coin Market Cap API
    
    # VARIABLES
    # Load the environment variables from the .env file
    load_dotenv('app.env')
    coin_market_api_key = os.getenv('COIN_MARKET_API_KEY')

    # Getting the necessary URL
    url = f'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?limit={1000}'


    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY':coin_market_api_key,
    } 

    session = Session()
    session.headers.update(headers)

    try:
      #response = session.get(url, params=parameters)
      response = session.get(url, headers=headers)
      data = json.loads(response.text)
      #print(json.dumps(data, indent=4, sort_keys=True))
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)


    coin_data = pd.DataFrame.from_dict((data["data"]))
    crypto_df = coin_data

    # Format this data frame by removing some columns right away
    coins_dataset_df = crypto_df.copy()
    coins_dataset_df = coins_dataset_df.drop(columns=['slug','num_market_pairs','tags','self_reported_circulating_supply', 'self_reported_market_cap','last_updated'])


    coins_dataset_df = pd.concat([coins_dataset_df, coins_dataset_df["quote"].apply(pd.Series)], axis=1)
    coins_dataset_df2 = pd.concat([coins_dataset_df, coins_dataset_df["USD"].apply(pd.Series)], axis=1)
    
    return(coins_dataset_df2)
    
    

## MAIN CODE:

In [ ]:
# 1. Need to know what coins
string = "-USD"
coin_list = ['BTC','ETH','ADA','BNB','XRP','LUNA']
coin_list = [s + string for s in coin_list]

coin_list2 = ['CRV','LDO','ANC','CVX','MKR']
coin_list2 = [s + string for s in coin_list]

### Getting The Data 

We chose 2018 to get complete data for all coins.

In [ ]:
# Get DataFrame of Chosen Coins
adj_close_df = cn_close_price_dfer(coin_list)

# Get DataFrame of Index/Indices
SPCBDM_path = Path("../Crypto_Trading/Resources/PerformanceGraphExport.xls")
spcbdm_indx = indx_close_price_xls(SPCBDM_path)

spcbdm_indx = spcbdm_indx['2018-01-01':'2022-05-06']
# Concatenate the coin and index dataframes into one df
concat_adj_close_df = pd.concat([adj_close_df, spcbdm_indx], axis=1)

concat_adj_close_df= concat_adj_close_df.dropna()
display(concat_adj_close_df.head())
display(concat_adj_close_df.tail())

### Gearing up for Analysis 

In [ ]:
# Variables
yearly_trading_days = 365 # For Crypto

# Data Frames Needed
#    1. Daily Returns
#    2. Average Annual Returns
#    3. Standard Deviation of Daily Returns
#    4. Sharpe Ratios
#    5. Correlation 


# Daily Returns
# Log of percentage change
df_daily_returns = concat_adj_close_df.pct_change()
df_daily_returns = df_daily_returns.dropna()

# Avg Annual Returns
avg_annual_return = df_daily_returns.mean() * yearly_trading_days
avg_annual_return.sort_values()



# Daily Returns STDDEV
df_daily_returns_stddev = df_daily_returns.std() * np.sqrt(yearly_trading_days)
df_daily_returns_stddev.sort_values()



#Sharpe Ratios
sharpe_ratios = avg_annual_return/df_daily_returns_stddev
sharpe_ratios.sort_values()


# Sharpe Graphs
sharpe_ratios.sort_values().hvplot.bar(
    hover_color='pink',
    title="Coin Portfolio & SPCBDM Sharpe Ratios"
).opts(
    width=700,
    yformatter='%.0f',
    #color="yellow",
    bgcolor="yellow",
    fontsize={
        'title': 15, 
        'labels': 12, 
    'xticks': 10, 
    'yticks': 10,},
    padding=0.1
)

## Brief Note on Correlation

In [ ]:
corr_df = concat_adj_close_df.corr(method='pearson')

#reset symbol as index (rather than 0-X)
corr_df.head().reset_index()

#del corr_df.index.name
corr_df.head(10)

plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(corr_df, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap of 6 Cryptocurrencies', fontdict={'fontsize':12}, pad=12);





## Asset Allocation: Weighing Strategies

### Weighting strategies
1. Market Cap Weighted - funds are distributed to each constituent based on each individual asset’s contribution to the sum of all asset’s market caps in the fund
2. Square Root Market Cap Weighted - than directly summing the market caps of each asset, the square root of the assets market cap is summed
3. *Evenly Weighted - evenly weight each asset in the portfolio. Each asset having the exact same value of funds allocated.
4. Minimum Weight - a maximum percent which can be allocated to a single asset, the remaining assets will have their allocations determined based on their market cap
5. Maximum Weight - a minimum percent which can be allocated to a single asset, the remaining assets will have their allocations determined based on their market cap

NOTE:
* Evenly Weighted: - some recent research has found evenly allocated indexes have tended to outperform market cap weighted indexes historically LINK [https://blog.shrimpy.io/blog/optimizing-asset-distribution-for-cryptocurrency-rebalancing]

The cryptocurrency market is still dominated by a few major players. The top two assets alone comprise 75% of the market cap. The tenth asset only holds .5% of the 
market cap. This vast disparity can lead to a lack of diversification in an index fund when allocating by market cap. We can use minimal and maximal weight allocations
to adjust for this:
    1. setting a minimum threshold of 5% for a top 10 market cap index fund. Each asset that would have held less than 5% of the weight in the fund will get bumped to 5%. 
    The remaining allocations will then be allocated based on market cap to distribute the funds.
    2. setting a maximum threshold of 25% on a top 10 market cap index fund. Any asset which holds over 25% of the market cap to 25%. Once capped, the remaining assets 
    will have their allocations determined based on their market cap.

In [ ]:
# Bring In Dataframe with marketcap data for our coins
# Using the Coin Market Cap API

# Varibles and values to be used
crypto_list = ['BTC','ETH','ADA','BNB','XRP','LUNA']
coin_nums = len(crypto_list)
eq_val =1/(coin_nums)

# Main DataFrame of our market cap information
# Calling the `mrkt_cap_api_df` function, and filtering out superfluous columns
mrk_cap_df = mrkt_cap_api_df()
mrk_cap_df = mrk_cap_df.drop(columns=['platform','volume_24h','volume_change_24h','percent_change_1h','quote','USD','percent_change_24h','percent_change_7d','percent_change_30d','percent_change_60d','percent_change_90d',
 'fully_diluted_market_cap','last_updated'])

# Filtered main market cap information with just the coins we need 
# Calculate important sums:
#.    1. sum coin market caps
#.    2. sum of the square root of the coin market caps

coin_mrk_cap_df = mrk_cap_df[mrk_cap_df.symbol.isin(crypto_list)]



# Adding some extra columns to the dataframe
coin_mrkcp_df = coin_mrk_cap_df.copy()

sum_coin_mrktcaps = coin_mrk_cap_df['market_cap'].sum()
coin_mrkcp_df["mk_weight"] = coin_mrk_cap_df["market_cap"]/sum_coin_mrktcaps


coin_mrkcp_df["sqrt_mk"] = np.sqrt(coin_mrk_cap_df['market_cap'])
sum_coin_mrktcaps_sqrt = coin_mrkcp_df["sqrt_mk"].sum()

coin_mrkcp_df["sqrt_mk_weight"] = coin_mrkcp_df["sqrt_mk"]/sum_coin_mrktcaps_sqrt
coin_mrkcp_df["eq_mk_weight"] = eq_val

# Checking what that df does
display(coin_mrkcp_df.head(6))




## Back Testing

### Now we have our weights
Portfolio backtesting seeks to determine the effectiveness of a trading model using historical data. We are going to use historic data with the assumption of a $1000
invested in the portfolio starting from 1/1/2018 to 5/6/2022. We are going to look at the calculated the cumulative return of the portfolio and compared it to the SPCBDM benchmark.

In [ ]:
# Variables: $ $ $ bills yah
# -------------------------
dollars_invested = 1000

## Variables: three different weights
eq_val_wght =coin_mrkcp_df["eq_mk_weight"].to_list()
mktcp_val_wght = coin_mrkcp_df["sqrt_mk_weight"].to_list()
sqrt_mktcp_val_wght = coin_mrkcp_df["mk_weight"].to_list()


coins_dly_rtrns = df_daily_returns[['BTC-USD','ETH-USD','ADA-USD','BNB-USD','XRP-USD','LUNA-USD']]
index_dly_rtrns = df_daily_returns[['SPCBDM-USD']]



# asset weights
wts1 = eq_val_wght
weighted_return = (wts1 * coins_dly_rtrns)

cumprod_portfolio = (1 + weighted_return).cumprod()
cumprod_index = (1 + index_dly_rtrns).cumprod()

df_cumlt_rtns= pd.concat([cumprod_portfolio, cumprod_index], axis=1)

df_cumlt_rtns2= df_cumlt_rtns.copy()


plot1 = df_cumlt_rtns.hvplot(
    x='Date',
    y=['BTC-USD','ETH-USD','ADA-USD','BNB-USD','XRP-USD','SPCBDM-USD'], value_label='Cumulative Returns',
    title="portfolio vs index without LUNA"
).opts(
    width=725,
    bgcolor="white",
    yformatter='%.0f',
    legend_position='right',
    legend_offset=(10, 50)
)


plot2 = df_cumlt_rtns2.hvplot(
    x='Date',
    y=['BTC-USD','ETH-USD','ADA-USD','BNB-USD','XRP-USD','LUNA-USD','SPCBDM-USD'], value_label='Cumulative Returns',
    title="portfolio vs index with LUNA"
).opts(
    width=725,
    bgcolor="white",
    yformatter='%.0f',
    legend_position='right',
    legend_offset=(10, 50)
)


display(plot1)
display(plot2)